In [1]:
import json
import pandas as pd
import numpy as np
import requests

ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"

query = {"instruments": "EUR_USD"} 
headers = {"Authorization": "Bearer "+ API_TOKEN}

response = requests.get("https://"+API_URL+PRICING_PATH, headers=headers, params=query) 

response.json()

{'time': '2024-08-04T22:42:45.879210140Z',
 'prices': [{'type': 'PRICE',
   'time': '2024-08-04T22:42:44.057145538Z',
   'bids': [{'price': '1.09049', 'liquidity': 500000},
    {'price': '1.09048', 'liquidity': 2500000},
    {'price': '1.09047', 'liquidity': 2000000},
    {'price': '1.09046', 'liquidity': 5000000},
    {'price': '1.09043', 'liquidity': 10000000},
    {'price': '1.09040', 'liquidity': 10000000}],
   'asks': [{'price': '1.09065', 'liquidity': 500000},
    {'price': '1.09066', 'liquidity': 500000},
    {'price': '1.09067', 'liquidity': 2000000},
    {'price': '1.09068', 'liquidity': 2000000},
    {'price': '1.09069', 'liquidity': 5000000},
    {'price': '1.09071', 'liquidity': 10000000},
    {'price': '1.09074', 'liquidity': 10000000}],
   'closeoutBid': '1.09040',
   'closeoutAsk': '1.09074',
   'status': 'tradeable',
   'tradeable': True,
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'EUR_USD'}]}

In [ ]:
import time

from_time = time.mktime(pd.to_datetime("01/01/2019").timetuple())
to_time = time.mktime(pd.to_datetime("01/02/2019").timetuple())


print(f"Window start: {from_time}, window end: {to_time}." )

header = {"Authorization": "Bearer "+API_TOKEN} 
query = {"from": str(from_time), "to": str(to_time), "granularity": "M1"}


INSTRUMENT = "GBP_JPY"

CANDLES_PATH = f"/v3/accounts/{ACCOUNT_ID}/instruments/{INSTRUMENT}/candles"


response = requests.get("https://"+API_URL+CANDLES_PATH, headers=header, params=query)


def json_to_pandas(json):
    json_file = json.json()
    price_json = json_file["candles"]
    dateTimes = []
    close_price, high_price, low_price, open_price= [], [], [], []
    volumes = []

    for candle in price_json:
        volumes.append(candle["volume"])
        dateTimes.append(candle["time"])
        close_price.append(float(candle["mid"]["c"]))
        high_price.append(float(candle["mid"]["h"]))
        low_price.append(float(candle["mid"]["l"]))
        open_price.append(float(candle["mid"]["o"]))
        
    dataFrame = pd.DataFrame({"DateTime": dateTimes,
                              "Open": open_price,"High": high_price,
                              "Low": low_price,"Close": close_price,
                              "Volume": volumes})
    dataFrame['DateTime'] = pd.to_datetime(dataFrame['DateTime'])
    dataFrame.set_index('DateTime', inplace=True)

    return dataFrame

GBPJPY_df = json_to_pandas(response)

GBPJPY_df.head()

In [ ]:
from OandaAPI import OandaAPI
oa = OandaAPI(account_id=ACCOUNT_ID, api_token=API_TOKEN)

gbp_jpy_df = oa.show_granularity()

gbp_jpy_df

In [32]:
ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"


def test_get_historical_data(instrument, start_date, end_date, granularity='D' ,as_pandas=True):
        # Must import Necessary Libraries
        import time
        import requests
        import pandas as pd

        # Convert Dates to DateTime objects
        start_dateTime = pd.to_datetime(start_date)
        end_dateTime = pd.to_datetime(end_date)

        # Convert Start Date & End Date to a UNIX timestamp
        unix_start = time.mktime(start_dateTime.timetuple())
        unix_end = time.mktime(end_dateTime.timetuple())

        # Authorization Header
        auth_header = {"Authorization": "Bearer "+API_TOKEN} 

        # Query Parameters
        parameters = {"from": str(unix_start), 
                      "to": str(unix_end), 
                      "granularity": granularity}
        
        # Historical Candle Stick Data URL
        candle_url = f"/v3/accounts/{ACCOUNT_ID}/instruments/{instrument}/candles"

        # Request for Candle Stick Data
        json_response = requests.get("https://"+API_URL+candle_url, headers=auth_header, params=parameters).json()

        # Return as Pandas DataFrame
        if as_pandas is True:
            # Convert Json to Pandas df
            candle_df = pd.json_normalize(json_response["candles"])

            # Drop complete Column
            candle_df.drop('complete', inplace=True, axis=1)

            # Rename Columns
            candle_df.rename(inplace=True,columns={'time':'DateTime',
                                                   'volume':'Volume',
                                                   'mid.o':'Open',
                                                   'mid.h':'High',
                                                   'mid.l':'Low',
                                                   'mid.c':'Close'})

            # Convert Datetime= Column to datetime object
            candle_df['DateTime'] = pd.to_datetime(candle_df['DateTime'])

            # Set DateTime to Index
            candle_df.set_index('DateTime',inplace=True)

            # Restructure Column Names
            candle_df = candle_df[['Open','High','Low','Close','Volume']]

            return candle_df
        else: 
            return json_response

df = test_get_historical_data(instrument='EUR_USD', 
                            start_date='08/01/2023',
                            end_date='08/01/2024',
                            granularity='H2')
df

,Open,High,Low,Close,Volume
DateTime,,,,,
2023-08-01 05:00:00+00:00,1.09904,1.10004,1.09868,1.09948,7560
2023-08-01 07:00:00+00:00,1.09944,1.09949,1.09702,1.09748,11225
2023-08-01 09:00:00+00:00,1.09749,1.09847,1.09658,1.09822,8147
2023-08-01 11:00:00+00:00,1.09822,1.09824,1.09524,1.09565,10231
2023-08-01 13:00:00+00:00,1.09566,1.09892,1.09560,1.09783,16453
...,...,...,...,...,...
2024-07-31 19:00:00+00:00,1.08265,1.08329,1.08170,1.08264,10399
2024-07-31 21:00:00+00:00,1.08248,1.08310,1.08241,1.08243,906
2024-07-31 23:00:00+00:00,1.08246,1.08301,1.08218,1.08277,7084


In [85]:
import time    
from datetime import datetime, timedelta
import math
def get_all_candles(start,end,gran):

    gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
                 'M1': 60,'M2': 120,'M4': 240,'M5': 300,
                 'M10': 600,'M15': 900,'M30': 1800,
                 'H1': 3600,'H2': 7200,'H3': 10800,
                 'H4': 14400,'H6': 21600,'H8': 28800,
                 'H12': 43200,'D': 86400,'W': 604800,
                 'M': 2620800}

    print(f'start Date:{start} & end Date:{end}')
    start_dateTime = pd.to_datetime(start)
    end_dateTime = pd.to_datetime(end)
    print(f'start DateTime:{start_dateTime} & end DateTime:{end_dateTime}')


    time_distance = end_dateTime - start_dateTime   
    seconds_between = time_distance.total_seconds() 
    num_data_points = seconds_between / gran_secs.get(gran)
    print(f'Seconds Between:{seconds_between}')


    if num_data_points > 4000:
        print('Greater than 4000 seconds')
        max_date = start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))
        print(f'First Max date: {max_date}')

        # Create Pandas DataFrame
        can_df = pd.DataFrame()

        new_start_dateTime = start_dateTime
       
        while True:
            if max_date >= end_dateTime:
                print(f'{new_start_dateTime} - {end_dateTime}')
                new_df = test_get_historical_data(instrument='EUR_USD', 
                                            start_date=new_start_dateTime,
                                            end_date=end_dateTime,
                                            granularity=gran)
                can_df = pd.concat([can_df, new_df])
                break

            else:
                print(f'{new_start_dateTime} - {max_date}')
                new_df = test_get_historical_data(instrument='EUR_USD', 
                                                start_date=new_start_dateTime,
                                                end_date=max_date,
                                                granularity=gran)
                new_start_dateTime = max_date+timedelta(seconds=gran_secs.get(gran))
                max_date = new_start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))
                can_df = pd.concat([can_df, new_df])
                                        
        return can_df
    else: 
        print(f'{start_dateTime} - {end_dateTime}')
        can_df = test_get_historical_data(instrument='EUR_USD', 
                                    start_date=start_dateTime,
                                    end_date=end_dateTime,
                                    granularity=gran)

        return can_df

master_df = get_all_candles(start='08/01/2005',
                            end='08/01/2024',
                            gran='M5')

master_df

start Date:08/01/2005 & end Date:08/01/2024
start DateTime:2005-08-01 00:00:00 & end DateTime:2024-08-01 00:00:00
Seconds Between:599616000.0
Greater than 4000 seconds
First Max date: 2005-08-14 21:20:00
2005-08-01 00:00:00 - 2005-08-14 21:20:00
2005-08-14 21:25:00 - 2005-08-28 18:45:00
2005-08-28 18:50:00 - 2005-09-11 16:10:00
2005-09-11 16:15:00 - 2005-09-25 13:35:00
2005-09-25 13:40:00 - 2005-10-09 11:00:00
2005-10-09 11:05:00 - 2005-10-23 08:25:00
2005-10-23 08:30:00 - 2005-11-06 05:50:00
2005-11-06 05:55:00 - 2005-11-20 03:15:00
2005-11-20 03:20:00 - 2005-12-04 00:40:00
2005-12-04 00:45:00 - 2005-12-17 22:05:00
2005-12-17 22:10:00 - 2005-12-31 19:30:00
2005-12-31 19:35:00 - 2006-01-14 16:55:00
2006-01-14 17:00:00 - 2006-01-28 14:20:00
2006-01-28 14:25:00 - 2006-02-11 11:45:00
2006-02-11 11:50:00 - 2006-02-25 09:10:00
2006-02-25 09:15:00 - 2006-03-11 06:35:00
2006-03-11 06:40:00 - 2006-03-25 04:00:00
2006-03-25 04:05:00 - 2006-04-08 01:25:00
2006-04-08 01:30:00 - 2006-04-21 22:50:0

,Open,High,Low,Close,Volume
DateTime,,,,,
2005-08-01 05:00:00+00:00,1.21860,1.21860,1.21840,1.21860,22
2005-08-01 05:05:00+00:00,1.21850,1.21860,1.21850,1.21850,15
2005-08-01 05:10:00+00:00,1.21845,1.21850,1.21820,1.21840,34
2005-08-01 05:15:00+00:00,1.21830,1.21850,1.21820,1.21850,21
2005-08-01 05:20:00+00:00,1.21840,1.21850,1.21790,1.21790,56
...,...,...,...,...,...
2024-08-01 04:35:00+00:00,1.08329,1.08330,1.08310,1.08323,279
2024-08-01 04:40:00+00:00,1.08324,1.08324,1.08312,1.08313,330
2024-08-01 04:45:00+00:00,1.08312,1.08325,1.08310,1.08318,346


In [109]:
import time
from datetime import datetime, timedelta
import math
import tqdm
def get_all_candles_upgraded(start,end,gran, verbose=False, pair='EUR_USD'):

    gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
                 'M1': 60,'M2': 120,'M4': 240,'M5': 300,
                 'M10': 600,'M15': 900,'M30': 1800,
                 'H1': 3600,'H2': 7200,'H3': 10800,
                 'H4': 14400,'H6': 21600,'H8': 28800,
                 'H12': 43200,'D': 86400,'W': 604800,
                 'M': 2620800}

    MAX_AMOUNT = 4000

    start_dateTime = pd.to_datetime(start)
    end_dateTime = pd.to_datetime(end)
    print(f'Gathering Data {start_dateTime} - {end_dateTime}')


    time_distance = end_dateTime - start_dateTime   
    seconds_between = time_distance.total_seconds() 

    if seconds_between > MAX_AMOUNT:
        max_date = start_dateTime+timedelta(seconds=MAX_AMOUNT*math.floor(gran_secs.get(gran)))

        # Create Pandas DataFrame
        can_df = pd.DataFrame()
        start_dateTime_list = []
        end_dateTime_list = []

        new_start_dateTime = start_dateTime

        while True:
            if max_date >= end_dateTime:
                start_dateTime_list.append(new_start_dateTime)
                end_dateTime_list.append(end_dateTime)
                print(f'Last Line: {new_start_dateTime} - {end_dateTime}')
                break

            else:
                # print(f'{new_start_dateTime} - {max_date}')
                start_dateTime_list.append(new_start_dateTime)
                end_dateTime_list.append(max_date)
                
                new_start_dateTime = max_date+timedelta(seconds=gran_secs.get(gran))
                max_date = new_start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))

        for i in tqdm.tqdm(range(len(start_dateTime_list)),colour='GREEN', 
                           ncols=100,desc =f'Downloading {pair} {gran} Data'):

            new_df = test_get_historical_data(instrument=pair, 
                                    start_date=start_dateTime_list[i],
                                    end_date=end_dateTime_list[i],
                                    granularity=gran)
            
            can_df = pd.concat([can_df, new_df])
    return can_df

master_df = get_all_candles_upgraded(start='08/01/2005',
                                     end='08/01/2024',
                                     gran='M5',
                                     pair='EUR_USD')

master_df

Gathering Data 2005-08-01 00:00:00 - 2024-08-01 00:00:00
Last Line: 2024-07-24 06:55:00 - 2024-08-01 00:00:00


,Open,High,Low,Close,Volume
DateTime,,,,,
2005-08-01 05:00:00+00:00,1.21860,1.21860,1.21840,1.21860,22
2005-08-01 05:05:00+00:00,1.21850,1.21860,1.21850,1.21850,15
2005-08-01 05:10:00+00:00,1.21845,1.21850,1.21820,1.21840,34
2005-08-01 05:15:00+00:00,1.21830,1.21850,1.21820,1.21850,21
2005-08-01 05:20:00+00:00,1.21840,1.21850,1.21790,1.21790,56
...,...,...,...,...,...
2024-08-01 04:35:00+00:00,1.08329,1.08330,1.08310,1.08323,279
2024-08-01 04:40:00+00:00,1.08324,1.08324,1.08312,1.08313,330
2024-08-01 04:45:00+00:00,1.08312,1.08325,1.08310,1.08318,346


In [114]:
master_df.to_csv('EUR-USD-M5.csv')